In [1]:
#Import necessary modules.
from alpha_vantage.timeseries import TimeSeries
import pandas

In [2]:
class ScriptData:
    list_of_tickers = []

    def __init__(self):
        self.api_key = "QIFV2AZUGKAUQA96"

    def __getitem__(self, script):
        self.__setitem__(script)
        df = self.convert_intraday_data(script)
        return df

    def __setitem__(self, script):
        self.list_of_tickers.append(script)
        return self.list_of_tickers

    def __contains__(self, script):
        if script in self.list_of_tickers:
            return True
        else:
            return False

    def fetch_intraday_data(self, script):
        ts = TimeSeries(self.api_key)
        data = ts.get_intraday(script, interval="60min", outputsize="full")
        return data

    def convert_intraday_data(self, script):
        data = self.fetch_intraday_data(script)
        final_data = list(data)
        final_data.pop()
        index_list = []
        column_list = []
        row_list = []
        #Converting the tuples fetched to appropriate data for further usage by converting it to data which has appropriate usable values
        for i in final_data:
            for j, k in i.items():
                row_element_list = []
                for key, value in k.items():
                    row_element_list.append(float(value))
                    if len(column_list) >= 5:
                        pass
                    else:
                        column_list.append(key)
                row_list.append(row_element_list)
                index_list.append(j)

        df = pandas.DataFrame(row_list, index=index_list, columns=column_list)
        return df


script_data = ScriptData()

In [3]:
script_data.fetch_intraday_data("GOOGL")
script_data.convert_intraday_data("GOOGL")
script_data["GOOGL"]

,1. open,2. high,3. low,4. close,5. volume
2022-12-23 20:00:00,89.09,89.19,89.0500,89.05,4048.0
2022-12-23 19:00:00,89.02,89.09,89.0200,89.09,3848.0
2022-12-23 18:00:00,89.08,89.12,89.0100,89.01,11113.0
2022-12-23 17:00:00,89.23,89.23,89.0000,89.04,526797.0
2022-12-23 16:00:00,89.40,89.49,88.9725,89.21,3917908.0
...,...,...,...,...,...
2022-10-31 09:00:00,95.35,96.29,95.1000,95.24,97183.0
2022-10-31 08:00:00,95.33,95.59,95.3300,95.41,24574.0
2022-10-31 07:00:00,95.42,95.42,94.9700,95.28,25432.0
2022-10-31 06:00:00,95.45,95.46,95.3000,95.45,3656.0


In [4]:
script_data.fetch_intraday_data("AAPL")
script_data.convert_intraday_data("AAPL")
script_data["AAPL"]

,1. open,2. high,3. low,4. close,5. volume
2022-12-23 20:00:00,131.7500,131.7500,131.6500,131.6500,25134.0
2022-12-23 19:00:00,131.7700,131.7800,131.7100,131.7400,16826.0
2022-12-23 18:00:00,131.7500,131.7900,131.7100,131.7800,34486.0
2022-12-23 17:00:00,131.8600,131.9900,131.5900,131.7500,1160991.0
2022-12-23 16:00:00,131.7189,131.9990,131.2000,131.8600,9403398.0
...,...,...,...,...,...
2022-10-31 09:00:00,154.5253,154.5999,153.0119,153.3410,1031512.0
2022-10-31 08:00:00,154.4377,154.5674,154.1187,154.1586,204334.0
2022-10-31 07:00:00,154.3281,154.4776,154.1286,154.3480,48704.0
2022-10-31 06:00:00,154.2682,154.6471,154.2184,154.3281,54015.0


In [5]:
print("GOOGL" in script_data)

True


In [6]:
print("AAPL" in script_data)

True


In [7]:
print("NVDA" in script_data)

False


In [8]:
def indicator1(df, timeperiod):
    indicator_list = []
    element_list = []
    index_col = []
    j = 1
    for i in df[df.columns[3]]:
        sum = 0
        if j < timeperiod:
            indicator_list.append(None)
            element_list.append(i)
        else:
            element_list.append(i)
            for k in element_list:
                l = float(k)
                sum += l
            avg = sum / timeperiod
            indicator_list.append(avg)
            del element_list[0]
        index_col.append(j)

        j += 1

    final_df = pandas.DataFrame(indicator_list, index=index_col, columns=["indicator"])
    timestamp_column = df.index
    final_df.insert(0, "timestamp", timestamp_column)

    return final_df

In [9]:
indicator1(script_data["GOOGL"], timeperiod=5)

,timestamp,indicator
1,2022-12-23 20:00:00,NaN
2,2022-12-23 19:00:00,NaN
3,2022-12-23 18:00:00,NaN
4,2022-12-23 17:00:00,NaN
5,2022-12-23 16:00:00,89.080
...,...,...
613,2022-10-31 09:00:00,94.692
614,2022-10-31 08:00:00,94.864
615,2022-10-31 07:00:00,94.974
616,2022-10-31 06:00:00,95.191


In [10]:
class Strategy:
    def __init__(self, ticker):
        self.ticker = ticker

    def get_script_data(self):
        df = script_data[self.ticker]
        df1 = indicator1(script_data[self.ticker], timeperiod=2)
        return df, df1

    def get_signals(self):
        df, df1 = self.get_script_data()
        #Setting up data for further implementation

        close_data = df.iloc[:, 3]
        indicator_data = df1.iloc[:, 1]
        close_dict = close_data.to_dict()
        close_list = list(close_dict.values())
        close_timestamp_list = list(close_dict.keys())
        indicator_dict = indicator_data.to_dict()
        indicator_list = list(indicator_dict.values())
        similar_data = []
        similar_data_timestamp = []
        signal = []
        length_of_similar_data_list = 0
        i = 0
        #Generating list of timestamp and prices where close_data and indicator_data where similar
        while i < len(indicator_list):
            j = 0
            list_for_avg_elements = []
            length_of_similar_data_list = len(similar_data)
            while j <= i:
                list_for_avg_elements.append(close_list[j])
                if indicator_list[i] == close_list[j]:
                    similar_data.append(close_list[j])
                    similar_data_timestamp.append(close_timestamp_list[j])
                j += 1
            avg = sum(list_for_avg_elements) / len(list_for_avg_elements)
            #Creating signals based on avg of price till the point of data known
            if length_of_similar_data_list < len(similar_data):
                counter = length_of_similar_data_list
                while counter < len(similar_data):
                    if similar_data[counter] >= avg:
                        signal.append("SELL")
                    elif similar_data[counter] < avg:
                        signal.append("BUY")
                    counter += 1
            i += 1
        #Creating signals for converting to data frame where no signals value is also included based upon appropriate timestamp
        signal_dict = dict(zip(similar_data, signal))
        final_signals = []
        for i, j in close_dict.items():
            if j in similar_data:
                if i in similar_data_timestamp:
                    final_signals.append(signal_dict.get(j))
                else:
                    final_signals.append("NO_SIGNAL")
            else:
                final_signals.append("NO_SIGNAL")

        signal_df = pandas.DataFrame(final_signals, index=close_data.index, columns=["signal"])
        final_signals_df = signal_df.loc[signal_df['signal'].isin(["BUY", "SELL"])]

        return final_signals_df

In [11]:
strategy = Strategy("NVDA")
strategy.get_script_data()
strategy.get_signals()

,signal
2022-12-21 11:00:00,SELL
2022-12-14 06:00:00,SELL
2022-12-08 18:00:00,SELL
2022-12-08 17:00:00,SELL
2022-12-07 10:00:00,BUY
2022-12-07 09:00:00,BUY
2022-12-02 05:00:00,SELL
2022-11-30 06:00:00,BUY
2022-11-30 05:00:00,BUY
2022-11-28 17:00:00,BUY
